In [ ]:
from reversegeocoding import ReverGeoCoder
from db_connector import DBConnector
from location_processor import LocationProcessor
from common import *
from symptom_processor import SymptomProcessor

from datetime import datetime
from sklearn.metrics import roc_auc_score, plot_roc_curve

import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)


import mysql.connector
import seaborn as sns
from datetime import date
from datetime import datetime
from datetime import timedelta

from getpass import getpass
import os
import glob
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim

import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from itertools import cycle
from tqdm.notebook import tqdm
import re
from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve

import matplotlib.pyplot as plt
import seaborn
from collections import defaultdict
import shap

from sklearn import svm

In [ ]:
geo_coder = ReverGeoCoder()
db_connector = DBConnector()
location_processor = LocationProcessor()

In [ ]:
df_pcr = db_connector.get_df("SELECT * FROM shine2_general_pcr")

In [ ]:
df_pcr = location_processor.limit_to_korea_location(df_pcr)

In [ ]:
df_pcr_loc = geo_coder.query_df(df_pcr)

### 샤인데이터에 들어있는 시군구 리스트 이름 뽑아와서 주소를 시군구로 바꿔주는 작업

In [ ]:
df_pcr_loc = location_processor.make_sidosigungu_columns(df_pcr_loc)

In [ ]:
df_pcr_loc.head()

### 이벤트 기간
- 1차: 2.28~4.4
- 2차: 8.16~10.30

### 여기서 일단 sigungu 있는 애들만 따로 저장

In [ ]:
df_pcr_loc = (df_pcr_loc[~((df_pcr_loc['sido'] == '') | (df_pcr_loc['sigungu'] == ''))])

In [ ]:
df_pcr_loc.head()

In [ ]:
df_pcr_loc[df_pcr_loc['pcr_result'] == '1'].shape[0], df_pcr_loc[df_pcr_loc['pcr_result'] == '2'].shape[0]

### baby 정보 가져오기

In [ ]:
df_kt_baby = db_connector.get_df('SELECT * FROM shine2_general_user')

In [ ]:
df_kt_baby = df_kt_baby[~df_kt_baby['gender'].isnull()]

In [ ]:
df_kt_baby['birthday'] = df_kt_baby['birthday'].astype(str)

In [ ]:
df_kt_baby['birthday'].apply(lambda x : x[:4]).unique()

In [ ]:
df_kt_baby['gender'] = df_kt_baby['gender'].astype(int)

In [ ]:
df_kt_baby['age'] = df_kt_baby['birthday'].apply(lambda x: datetime.now().year - int(x[:4]))

In [ ]:
df_kt_baby['age_60_and_above'] = df_kt_baby['age'].apply(lambda age: 1 if age >= 60 else 0)

In [ ]:
df_kt_baby[df_kt_baby['age_60_and_above'] == 1]

### 시군구 넣어준 md_data에서 pcr 데이터랑 selfcheck 데이터 따로 뽑기

In [ ]:
patients_str = ','.join(df_pcr_loc['patient_id'].astype(str).unique().tolist())

In [ ]:
df_selfcheck = db_connector.get_df(f"SELECT * FROM shine2_general_selfcheck WHERE patient_id IN ({patients_str})")

In [ ]:
df_selfcheck['reg_date'].astype(str)

In [ ]:
df_pcr_loc.shape[0]

In [ ]:
df_pcr_loc[df_pcr_loc['pcr_result'] == '2'].shape[0], df_pcr_loc[df_pcr_loc['pcr_result'] == '1'].shape[0]

In [ ]:
df_pcr_loc['patient_id'].shape[0], df_pcr_loc['patient_id'].unique().shape[0]

In [ ]:
df_pcr_loc.shape[0]

In [ ]:
df_kt_baby['patient_id'] = df_kt_baby['patient_id'].astype(int)

### pcr 데이터에 유저 정보(생일, 성별, 나이, 60세이상) 입력 + pcr에 붙은 sido, sigungu 살림

In [ ]:
df_pcr_loc = pd.merge(df_pcr_loc, df_kt_baby, how='left', on='patient_id')

In [ ]:
df_pcr_loc.columns

In [ ]:
df_pcr_loc['pcr_date'] = df_pcr_loc['pcr_date'].astype(str)
df_selfcheck['selfcheck_date'] = df_selfcheck['selfcheck_date'].astype(str)

### pcr 데이터 입력 당시 시군구를 넣음

In [ ]:
df_res_temp = pd.merge(df_pcr_loc, df_selfcheck, how='inner', on='patient_id', suffixes=('_pcr', '_selfcheck'))

In [ ]:
df_res_temp.columns

In [ ]:
df_res_temp['time_diff'] = df_res_temp.apply(lambda x: generate_diff_date(x['selfcheck_date'], x['pcr_date']), axis=1)

In [ ]:
df_res_temp = df_res_temp.sort_values('time_diff')
df_res_temp = df_res_temp.drop_duplicates(subset='_id_pcr', keep="first")

In [ ]:
df_res_temp = df_res_temp[df_res_temp['time_diff'] <= 14]

In [ ]:
df_res_temp.head()

In [ ]:
df_res = df_res_temp[['patient_id', 'gender', 'age', 'birthday', 'age_60_and_above',\
                                   'fever', 'sx_name', 'etc_status', 'is_abroad', 'is_visit', \
                                   'is_contact', 'is_isolation', 'selfcheck_reason',\
                                   'selfcheck_date', 'pcr_date', 'pcr_place', 'sido', 'sigungu', 'pcr_result']]

In [ ]:
df_res['fever'] = df_res['fever'].apply(lambda x: 0 if x == '' else x)

In [ ]:
symptom_processor = SymptomProcessor()

In [ ]:
df_res = symptom_processor.make_symptom_columns(df_res)

In [ ]:
df_res['weekday'] = pd.to_datetime(df_res['pcr_date']).dt.dayofweek

In [ ]:
df_res['pcr_result'] = df_res['pcr_result'].astype(int)

In [ ]:
df_res['age_ratio'] = df_res['age'] / 100

In [ ]:
df_res[df_res['pcr_result'] == 1].shape[0], df_res[df_res['pcr_result'] == 2].shape[0]

In [ ]:
df_res = df_res.sort_values('pcr_date', ascending=True)

In [ ]:
df_res['pcr_result'] = df_res['pcr_result'].apply(lambda x: x-1)

In [ ]:
df_res['pcr_result'] = df_res['pcr_result'].astype(int)

In [ ]:
df_res['age_ratio'] = df_res['age'] / 100

In [ ]:
df_res[df_res['pcr_result'] == 0].shape[0], df_res[df_res['pcr_result'] == 1].shape[0]

In [ ]:
df_res.columns

In [ ]:
df_res['pcr_date'].max()

In [ ]:
df_res[df_res['pcr_date'] <= '2022-03-11'].shape

In [ ]:
df_covid = df_res

### 본격적으로 유행도 붙여주는 작업

In [ ]:
import surveillance_processor

In [ ]:
surveillance_processor = surveillance_processor.SurveillanceProcessor()

In [ ]:
surveillance_processor

In [ ]:
global_confirmed_df, kor_confirmed_df = surveillance_processor.get_confirmed_df()

In [ ]:
kor_confirmed_df.tail()

In [ ]:
global_confirmed_df.tail()

In [ ]:
CONFIRMED_MAX = 11934 / 9.217 * 51.78  # 66033
GLOBAL_CONFIRMED_MAX = 1498041

In [ ]:
kor_confirmed_df['confirmed_ratio'] = kor_confirmed_df['confirmed_num'] / CONFIRMED_MAX
global_confirmed_df['global_confirmed_ratio'] = global_confirmed_df['global_confirmed_num'] / GLOBAL_CONFIRMED_MAX
kor_confirmed_df['confirmed_week_ratio'] = kor_confirmed_df['confirmed_week_num'] / CONFIRMED_MAX
global_confirmed_df['global_confirmed_week_ratio'] = global_confirmed_df['global_confirmed_week_num'] / GLOBAL_CONFIRMED_MAX

In [ ]:
df_pcr_loc

In [ ]:
df_sigungu_population = df_pcr_loc[['sido', 'sigungu']].drop_duplicates().reset_index(drop=True)

In [ ]:
df_sigungu_population

In [ ]:
df_sigungu_population['population'] = df_sigungu_population.apply(lambda x: location_processor.find_population(x['sido'], x['sigungu']), axis=1)

In [ ]:
df_sigungu_population = df_sigungu_population[~df_sigungu_population['population'].isnull()].reset_index(drop=True)

### 시도시군구 인구수 구함

In [ ]:
df_sigungu_population.sort_values(['sido', 'sigungu'])

In [ ]:
def find_sigungu_confirmed_ratio(sido, sigungu, confirmed_total):
    sido_condit = df_sigungu_population['sido'] == sido 
    sigungu_condit = df_sigungu_population['sigungu'] == sigungu
    df_popul = df_sigungu_population[sido_condit & sigungu_condit]
    if df_popul.shape[0] == 0: return None
    return float(confirmed_total) / df_popul['population'].item()

### 우리 지역별 확진자수 디비에서 가져와서 csv파일에 저장한거 불러옴

In [ ]:
# df_confirmed_sigungu_old = pd.read_csv('sigungu_confirmed_num.csv')

In [ ]:
df_confirmed_sigungu = db_connector.get_df('SELECT * FROM shine2_covid_sigungu')

In [ ]:
df_confirmed_sigungu

In [ ]:
df_confirmed_sigungu['confirmed_total'] = df_confirmed_sigungu['confirmed_total'].apply(commanum_revert_to_int)

In [ ]:
df_confirmed_sigungu['standard_date'] = df_confirmed_sigungu['standard_date'].astype(str)

### 기존에 시군구 다 존재하던 애들에 sigungu_confirmed_ratio 붙여줌

In [ ]:
df_confirmed_sigungu['sigungu_confirmed_ratio'] = df_confirmed_sigungu.apply(lambda x : find_sigungu_confirmed_ratio(x['sido'], x['sigungu'], x['confirmed_total']), axis=1)

In [ ]:
df_confirmed_sigungu = df_confirmed_sigungu[~df_confirmed_sigungu['sigungu_confirmed_ratio'].isnull()]

In [ ]:
df_confirmed_sigungu = df_confirmed_sigungu.rename(columns={'standard_date': 'pcr_date'})

In [ ]:
df_confirmed_sigungu['pcr_date'] = df_confirmed_sigungu['pcr_date'].astype(str).apply(lambda x : x[:10])

### 전세계 유행도, 한국 유행도 붙임

In [ ]:
df_covid = pd.merge(df_covid, kor_confirmed_df, how='left', left_on='pcr_date', right_on='confirmed_date')

In [ ]:
df_covid = pd.merge(df_covid, global_confirmed_df, how='left', left_on='pcr_date', right_on='global_confirmed_date')

In [ ]:
df_covid.head()

### 시군구 유행도 붙임

In [ ]:
df_confirmed_sigungu

In [ ]:
df_covid = pd.merge(df_covid, df_confirmed_sigungu, on=['pcr_date', 'sido', 'sigungu'], how='left')

In [ ]:
display(df_confirmed_sigungu)
display(df_covid)

In [ ]:
df_covid[df_covid['sigungu_confirmed_ratio'].isnull()].shape, df_covid[~df_covid['sigungu_confirmed_ratio'].isnull()].shape

In [ ]:
df_covid['sigungu_confirmed_ratio'].unique()

In [ ]:
CONFIRMED_MAX = 11934 / 9.217 * 51.78  # 66033
GLOBAL_CONFIRMED_MAX = 1498041

In [ ]:
df_covid['confirmed_ratio'] = df_covid['confirmed_num'] / CONFIRMED_MAX
df_covid['global_confirmed_ratio'] = df_covid['global_confirmed_num'] / GLOBAL_CONFIRMED_MAX
df_covid['confirmed_week_ratio'] = df_covid['confirmed_week_num'] / CONFIRMED_MAX
df_covid['global_confirmed_week_ratio'] = df_covid['global_confirmed_week_num'] / GLOBAL_CONFIRMED_MAX

In [ ]:
df_covid = df_covid[~df_covid['confirmed_date'].isna()]

### 시군구 있는 데이터 없는 데이터 구분하는 마스킹 컬럼 추가

In [ ]:
def mask_filter(sido, sigungu):
    if sido == '' or sigungu == '': return 0
    return 1

In [ ]:
df_covid['mask'] = df_covid.apply(lambda x: mask_filter(x['sido'], x['sigungu']), axis=1)

In [ ]:
df_covid['sigungu_confirmed_ratio'] = df_covid['sigungu_confirmed_ratio'].fillna(0)

In [ ]:
df_covid.columns

In [ ]:
israel_symptom_input_cols = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache']

symptom_input_cols = (['runny_nose', 'muscle_pain', 'chills',  'loss_of_taste', 
                       'loss_of_smell', 'sputum', 'chest_pain'])

surveilance_cols = (['global_confirmed_ratio', 'confirmed_ratio', 'sigungu_confirmed_ratio'])
# surveilance_cols = (['global_confirmed_ratio', 'confirmed_ratio'])

# shap value 찍어보자!! kernel explain

mask_col = ['mask']

indication_cols = ['indication_other','indication_abroad', 'indication_contact']
other_cols = ['gender', 'age_ratio', 'weekday']
res_col = ['pcr_result']

# cols = (israel_symptom_input_cols + symptom_input_cols + 
#         indication_cols + surveilance_cols + mask_col + 
#         other_cols + res_col)

israel_etc_input_cols = ['age_60_and_above', 'gender', 'indication_other','indication_abroad', 'indication_contact']
cols = israel_symptom_input_cols + israel_etc_input_cols + res_col

# cols = israel_symptom_input_cols + symptom_input_cols + surveilance_cols + mask_col + res_col


# symptom_input_cols = ['cough', 'fever', 'confirmed_ratio', \
#                       'sigungu_confirmed_ratio', 'mask', 'pcr_result']
#
# 'indication_other','indication_abroad', 'indication_contact'
#'mask', 'confirmed_ratio', 'sigungu_confirmed_ratio'
# 'confirmed_week_ratio', 'global_confirmed_week_ratio', 

# 0~10 기존 / 11~17 추가된 7 feature

# input_cols = ['cough', 'fever', 'sore_throat','shortness_of_breath',\
#               'head_ache', 'gender', 'runny_nose', 'muscle_pain', 'chills',\
#               'loss_of_taste', 'loss_of_smell', 'sputum', 'chest_pain', 'age_ratio', 'weekday'
#
#               'gender', 'confirmed_ratio',\
#               'global_confirmed_ratio', 'indication_other',\
#               'indication_abroad', 'indication_contact', 'pcr_result']

In [ ]:
input_cols = ['pcr_date'] + cols
# input_cols = cols

In [ ]:
df_input = df_covid[~df_covid['confirmed_ratio'].isna()]

### mask를 이용해서 시군구 있는 애들만 뽑는다 (실험 조건 중 하나)

In [ ]:
# df_input = df_input[df_input['mask'] == 1]

In [ ]:
df_input['pcr_result'].unique()

In [ ]:
neg_cnt = df_input[df_input['pcr_result'] == 0].shape[0]
neg_cnt

#### 이벤트 기간
1차: 2.28 - 4.4
2차: 8.16 - 10.31

In [ ]:
event_start_date_1 = '2022-02-14'
event_end_date_1 = '2022-04-18'
event_start_date_2 = '2022-08-02'
event_end_date_2 = '2022-11-14'

In [ ]:
# df_input_balanced = df_input[(event_start_date_2 <= df_input['test_date']) & (df_input['test_date'] <= event_end_date_2)]
df_input_balanced = df_input

In [ ]:
from collections import Counter

In [ ]:
baby_id_counter = Counter(df_input_balanced['patient_id'])

In [ ]:
baby_ids_only_one = [x for x, count in baby_id_counter.items() if count == 1]
len(baby_ids_only_one)

In [ ]:
baby_ids = df_input_balanced['patient_id'].unique()

In [ ]:
baby_ids.shape

In [ ]:
Counter(baby_id_counter.values())

In [ ]:
# df_input_balanced = df_input_balanced[df_input_balanced['baby_id'].isin(baby_ids_only_one)]

In [ ]:
# df_input_balanced

In [ ]:
# input_cols.append('baby_id')

In [ ]:
input_cols = ['patient_id'] + input_cols
input_cols

In [ ]:
df_input_final = df_input_balanced[input_cols]

In [ ]:
df_input_final.sort_values('pcr_date')

### 최종 데이터셋 음성 양성 갯수

In [ ]:
df_input_final[df_input_final['pcr_result'] == 0].shape, df_input_final[df_input_final['pcr_result'] == 1].shape

In [ ]:
def split_input_gt(non_split_data):
    splited_input = non_split_data[:, :-1]
    splited_gt = non_split_data[:, -1]
    
    return splited_input, splited_gt

In [ ]:
for col in input_cols:
    if df_input_final[df_input_final[col].isna()].shape[0] != 0:
        df_input_final[df_input_final[col].isna()] = df_input_final[df_input_final[col].isna()].fillna(0)

In [ ]:
np.random.seed(1212)

In [ ]:
df_train, df_test = split_df_with_ratio(df_input_final, 0.4)
df_test, df_vaild = split_df_with_ratio(df_test, 0.5)

In [ ]:
df_train['pcr_result'].value_counts()

In [ ]:
df_test['pcr_result'].value_counts()

In [ ]:
df_vaild['pcr_result'].value_counts()

In [ ]:
df_train.shape, df_test.shape, df_vaild.shape

In [ ]:
# df_train.to_csv('df_train_v3_1.csv', index=False)
# df_vaild.to_csv('df_valid_v3_1.csv', index=False)
# df_test.to_csv('df_test_v3_1.csv', index=False)